Sketchbook to experiment with RNN models to model the dynamics. Once tested, the code in this notebook will be incorporated into functions/classes/ python modules.

## Dynamics Model
Models the dynamics of the system p(next observation | history of observations, action)

History of observations consist of exercises a student has done and whether the student solved each of them

Action is the next exercise chosen 

Next observation is whether the student gets the chosen exercise correct


We want to use an RNN to model the dynamics.
Input data represents history of observations, of shape (n_students, n_timesteps, observation_vec_size)

Output represents the probability of getting next exercise correctly, of shape (n_students, n_timesteps, n_exercises) 

So at each timestep, we make a prediction for all actions. 

For each action, the output vector specifies the predicted probability of the student getting the chosen exercise correctly.

The target output only contains binary values.

In [16]:
import sys
print sys.executable
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

/Users/lisa1010/tf_venv/bin/python
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sonnet as snt
import tensorflow as tf
import tflearn
import numpy as np

In [3]:
import dataset_utils

Load toy data set

In [5]:
data = dataset_utils.load_data(filename="../synthetic_data/toy.pickle")
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

# Build RNN Model

In [6]:
tflearn.init_graph()

gpu_options {
}
allow_soft_placement: true

In [9]:
n_hidden = 64
n_samples, n_timesteps, n_inputdim = input_data_.shape
_,_,n_outputdim = target_data_.shape

In [10]:
print n_timesteps

50


In [12]:
print n_inputdim

20


In [13]:
print n_outputdim

10


In [15]:
graph_to_use = tf.Graph()
with graph_to_use.as_default():
    net = tflearn.input_data([None, n_timesteps, n_inputdim],dtype=tf.float32, name='input_data')
    output_mask = tflearn.input_data([None, n_timesteps, n_outputdim], dtype=tf.float32, name='output_mask')
    net = tflearn.lstm(net, n_hidden, return_seq=True, name="lstm_1")
    net = tflearn.lstm(net, n_outputdim, return_seq=True, name="lstm_2")
    net = tf.stack(net, axis=1)
    preds = net
    net = net * output_mask
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='mean_square')
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit([ input_data_, output_mask_], target_data_, validation_set=0.1)

---------------------------------
Run id: 2PTSBL
Log directory: /tmp/tflearn_logs/
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
---------------------------------
Training samples: 4
Validation samples: 1
--
Training Step: 1  | time: 2.116s
| Adam | epoch: 001 | loss: 0.00000 | val_loss: 0.04748 -- iter: 4/4
--
Training Step: 2  | total loss: 0.04031 | time: 1.067s
| Adam | epoch: 002 | loss: 0.04031 | val_loss: 0.04669 -- iter: 4/4
--
Training Step: 3  | total loss: 0.04343 | time: 1.065s
| Adam | epoch: 003 | loss: 0.04343 | val_loss: 0.04589 -- iter: 4/4
--
Training Step: 4  | total loss: 0.04341 | time: 1.061s
| Adam | epoch: 004 | loss: 0.04341 | val_loss: 0.04506 -- iter: 4/4
--
Training Step: 5  | total loss: 0.04293 | time: 1.060s
| Adam | epoch: 005 | loss: 0.04293 | val_los

In [72]:
tf.get_collection(tf.GraphKeys.INPUTS)

[<tf.Tensor 'output_mask:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_1:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData_1/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_2:0' shape=(?, 50, 10) dtype=int32>,
 <tf.Tensor 'InputData_2/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_3:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'InputData_3/X:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'input_data:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_4:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_1:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_5:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_2:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_6:0' shape=(?, 50, 10) dtype=float32>,
 <tf.Tensor 'input_data_3:0' shape=(?, 50, 20) dtype=float32>,
 <tf.Tensor 'output_mask_7:0' shape=(?, 50, 10) dtype=fl

In [17]:
data = dataset_utils.load_data(filename="../synthetic_data/1000stud_100seq_expert.pickle")
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [20]:
n_samples, n_timesteps, n_inputdim = input_data_.shape
_,_,n_outputdim = target_data_.shape
print n_samples
print n_timesteps
print n_inputdim
print n_outputdim

1000
100
20
10


In [22]:
graph_to_use = tf.Graph()
with graph_to_use.as_default():
    net = tflearn.input_data([None, n_timesteps, n_inputdim],dtype=tf.float32, name='input_data')
    output_mask = tflearn.input_data([None, n_timesteps, n_outputdim], dtype=tf.float32, name='output_mask')
    net = tflearn.lstm(net, n_hidden, return_seq=True, name="lstm_1")
    net = tflearn.lstm(net, n_outputdim, return_seq=True, name="lstm_2")
    net = tf.stack(net, axis=1)
    preds = net
    net = net * output_mask
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='mean_square')
    model = tflearn.DNN(net, tensorboard_verbose=2)
    model.fit([ input_data_, output_mask_], target_data_, n_epoch=32, validation_set=0.1)

Training Step: 479  | total loss: 0.02480 | time: 2.358s
| Adam | epoch: 032 | loss: 0.02480 -- iter: 896/900
Training Step: 480  | total loss: 0.02479 | time: 3.469s
| Adam | epoch: 032 | loss: 0.02479 | val_loss: 0.02476 -- iter: 900/900
--


In [23]:
from dynamics_model import *

In [39]:
model = load_model(model_id="test_model", load_checkpoint=False, is_training=True)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model/
Checkpoint directory path: ../checkpoints/test_model/
Model loaded.


In [40]:
train_data = (input_data_[:,:10,:], output_mask_[:,:10,:], target_data_[:,:10,:])

In [41]:
train(model, train_data)

Training Step: 44  | total loss: 0.03538 | time: 0.681s
| Adam | epoch: 003 | loss: 0.03538 -- iter: 896/900
Training Step: 45  | total loss: 0.03482 | time: 1.726s
| Adam | epoch: 003 | loss: 0.03482 | val_loss: 0.03224 -- iter: 900/900
--


KeyboardInterrupt: 

In [43]:
predict(model, input_data_[:10, :10, :])

[[[-0.007385450880974531,
   0.025550365447998047,
   0.011329921893775463,
   0.03704448416829109,
   0.039401981979608536,
   0.027618983760476112,
   0.017678581178188324,
   0.025421805679798126,
   0.01569029688835144,
   -0.02289711870253086],
  [-0.014513563364744186,
   0.0470220223069191,
   0.040145840495824814,
   0.09161123633384705,
   0.10280773788690567,
   0.06935103237628937,
   0.05226275697350502,
   0.041719403117895126,
   0.04879506677389145,
   -0.07392476499080658],
  [-0.03218340128660202,
   0.07135201245546341,
   0.0685245469212532,
   0.1436130404472351,
   0.19665291905403137,
   0.1438368409872055,
   0.1049000471830368,
   0.06255870312452316,
   0.1026017889380455,
   -0.11450488120317459],
  [-0.06921423971652985,
   0.08478009700775146,
   0.1043052151799202,
   0.19068250060081482,
   0.2767428159713745,
   0.23720687627792358,
   0.17555317282676697,
   0.10750668495893478,
   0.16081872582435608,
   -0.14396199584007263],
  [-0.11939936876296997,
 

In [52]:
import dynamics_model_class as dm

In [53]:
dmodel = dm.DynamicsModel(model_id="test_model",  load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model/
Checkpoint directory path: ../checkpoints/test_model/
Model loaded.


In [56]:
dmodel.train(train_data)

Training Step: 1019  | total loss: 0.02487 | time: 0.567s
| Adam | epoch: 068 | loss: 0.02487 -- iter: 896/900


KeyboardInterrupt: 

In [57]:
dmodel.predict(input_data_[:10,:10, :])

[[[0.49599671363830566,
   0.505977988243103,
   0.49741360545158386,
   0.5000333189964294,
   0.498515248298645,
   0.5003623366355896,
   0.5020152926445007,
   0.4968193471431732,
   0.5021724104881287,
   0.49825751781463623],
  [0.48964592814445496,
   0.5083518624305725,
   0.49532902240753174,
   0.5052512884140015,
   0.49550819396972656,
   0.5047191381454468,
   0.49943220615386963,
   0.49179941415786743,
   0.5016341805458069,
   0.4988301992416382],
  [0.48247480392456055,
   0.5100704431533813,
   0.4926689863204956,
   0.5104095339775085,
   0.4944813549518585,
   0.5111148357391357,
   0.49826157093048096,
   0.4853592813014984,
   0.4960009455680847,
   0.49970728158950806],
  [0.4703198969364166,
   0.5135535001754761,
   0.4915870130062103,
   0.5127080678939819,
   0.49413204193115234,
   0.5221576690673828,
   0.502672016620636,
   0.47870734333992004,
   0.4900960922241211,
   0.5019947290420532],
  [0.4537630081176758,
   0.520048201084137,
   0.4903250336647033